In [27]:
import QuantLib as ql
from openbb_terminal.sdk import openbb
import pandas as pd

In [26]:
# Option chain discovery for SPY contracts. I narrowed down to a March 17 Call with a strike of 400 to find the IV and pricing of the option

spy_chain = openbb.stocks.options.chains(symbol = 'TXN', source='YahooFinance')
march_chain = spy_chain[spy_chain['expiration'] == '2023-03-24']
#march_chain = march_chain[march_chain['strike'] == ]
march_chain.head(50)


,strike,lastPrice_c,bid_c,ask_c,volume_c,openinterest_c,iv_c,lastPrice_p,bid_p,ask_p,volume_p,openinterest_p,iv_p,expiration
0,110.0,64.90,67.55,68.15,13.0,13.0,2.246098,NaN,NaN,NaN,NaN,NaN,NaN,2023-03-24
1,115.0,60.83,62.70,63.05,7.0,7.0,2.083989,NaN,NaN,NaN,NaN,NaN,NaN,2023-03-24
2,150.0,25.70,27.80,28.10,NaN,10.0,0.980469,0.07,0.01,0.03,5.0,67.0,0.566411,2023-03-24
3,160.0,15.30,17.80,18.15,2.0,6.0,0.681644,0.10,0.08,0.10,3.0,214.0,0.462896,2023-03-24
4,162.5,13.91,15.35,15.65,2.0,1.0,0.611332,0.14,0.13,0.14,70.0,103.0,0.428717,2023-03-24
5,165.0,12.10,13.00,13.30,4.0,30.0,0.567387,0.23,0.22,0.25,61.0,479.0,0.414557,2023-03-24
6,167.5,9.55,10.65,10.95,3.0,36.0,0.514165,0.51,0.36,0.40,2.0,647.0,0.391119,2023-03-24
7,170.0,8.75,8.35,8.60,5.0,55.0,0.476568,0.56,0.60,0.65,4.0,242.0,0.369147,2023-03-24
8,172.5,6.35,6.40,6.55,3.0,228.0,0.443365,0.94,1.01,1.05,86.0,874.0,0.347663,2023-03-24
9,175.0,4.80,4.55,4.65,81.0,415.0,0.406500,1.65,1.64,1.71,49.0,248.0,0.331306,2023-03-24


In [29]:
# Define option data to load into the model
maturity_date = ql.Date(24,3,2023)
spot_price = 177.69
strike_price = 180
iv = .353278	
dividend_rate = 0.0
option_type = ql.Option.Call

risk_free_rate = 0.001
day_count=ql.Actual365Fixed()
calendar = ql.NullCalendar()

calculation_date = ql.Date.todaysDate()
ql.Settings.instance().evaluationDate = calculation_date

payoff = ql.PlainVanillaPayoff(option_type, strike_price)
settlement = calculation_date 

am_exercise = ql.AmericanExercise(settlement, maturity_date)
american_option = ql.VanillaOption(payoff, am_exercise)

In [30]:
#Prep the Black_Scholes varibles for loading into BlackScholesMertonProcess

spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))

flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date,risk_free_rate,day_count))

divident_yield = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date,dividend_rate,day_count))

flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date,calendar,iv,day_count))

bsm_process = ql.BlackScholesMertonProcess(spot_handle,flat_ts,divident_yield,flat_vol_ts)




In [31]:
#Load Black Scholes process into the pricing engine to produce an expected value of the option premium 
steps = 200
binomial_engine = ql.BinomialVanillaEngine(bsm_process,"crr",steps)
american_option.setPricingEngine(binomial_engine)

spy_ev = american_option.NPV()
##spy_av = march_chain.loc[64:['bid_c']]

print(spy_ev)
#print(spy_av)

1.6450820771493988


In [12]:
price_diff = spy_ev - spy_av 

price_diff

52    -7.814223
53          NaN
54    -4.014223
55    -3.014223
56    -2.554223
57          NaN
58    -2.054223
59    -1.564223
60    -1.114223
61     0.235777
62    -0.674223
63     0.235777
64    -0.304223
65     0.235777
66    -0.034223
67     0.235777
68     0.115777
69     0.235777
70     0.195777
71     0.235777
72     0.225777
73     0.235777
74     0.235777
75     0.235777
76     0.235777
77     0.235777
78     0.235777
79     0.235777
80     0.235777
81     0.235777
82     0.235777
83     0.235777
84     0.235777
85     0.235777
86     0.235777
87     0.235777
88     0.235777
89     0.235777
90     0.235777
91     0.235777
92     0.235777
93     0.235777
94     0.235777
95     0.235777
96     0.235777
97          NaN
98          NaN
99          NaN
100         NaN
101         NaN
102         NaN
Name: bid_c, dtype: float64